# BEE 4750 Homework 5: Mixed Integer and Stochastic Programming

**Name**: Badyn Milstein-Touesnard, Isabel Camara, Mateo Aranzazu

**ID**: 5286287, 5273977, 5355552

**netID**: bmm98, ic277, ma956

> **Due Date**
>
> Thursday, 12/05/24, 9:00pm

## Overview

### Instructions

-   In Problem 1, you will use mixed integer programming to solve a
    waste load allocation problem.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `~/BEE 4750/hw5-ismabay`
   Installed Adapt ────────────── v4.1.1
   Installed StaticArrays ─────── v1.9.8
   Installed GeoInterface ─────── v1.3.8
   Installed ChainRulesCore ───── v1.25.0
   Installed NetworkLayout ────── v0.4.7
   Installed Graphs ───────────── v1.12.0
   Installed MutableArithmetics ─ v1.5.2
   Installed StructArrays ─────── v0.6.20
Precompiling project...
  ✓ Adapt
  ✓ GeoInterface
  ✓ EarCut_jll
  ✓ ChainRulesCore
  ✓ ChainRulesCore → ChainRulesCoreSparseArraysExt
  ✓ OffsetArrays → OffsetArraysAdaptExt
  ✓ LogExpFunctions → LogExpFunctionsChainRulesCoreExt
  ✓ ColorVectorSpace → SpecialFunctionsExt
  ✓ MutableArithmetics
  ✓ Colors
  ✓ SpecialFunctions → SpecialFunctionsChainRulesCoreExt
  ✓ StaticArrays
  ✓ Adapt → AdaptStaticArraysExt
  ✓ StaticArrays → StaticArraysStatisticsExt
  ✓ ConstructionBase → ConstructionBaseStaticArraysExt
  ✓ ForwardDiff → ForwardDiffStaticArraysExt
  ✓ StaticArrays → StaticArraysChainRulesCoreExt
  ✓ StructA

In [2]:
using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables

## Problems (Total: 30 Points)

### Problem 1 (30 points)

Three cities are developing a coordinated municipal solid waste (MSW)
disposal plan. Three disposal alternatives are being considered: a
landfill (LF), a materials recycling facility (MRF), and a
waste-to-energy facility (WTE). The capacities of these facilities and
the fees for operation and disposal are provided below.

-   **LF**: Capacity 200 Mg, fixed cost \$2000/day, tipping cost
    \$50/Mg;
-   **MRF**: Capacity 350 Mg, fixed cost \$1500/day, tipping cost
    \$7/Mg, recycling cost \$40/Mg recycled;
-   **WTE**: Capacity 210 Mg, fixed cost \$2500/day, tipping cost
    \$60/Mg;

The MRF recycling rate is 40%, and the ash fraction of non-recycled
waste is 16% and of recycled waste is 14%. Transportation costs are
\$1.5/Mg-km, and the relative distances between the cities and
facilities are provided in the table below.

| **City/Facility** | **Landfill (km)** | **MRF (km)** | **WTE (km)** |
|:-----------------:|:-----------------:|:------------:|:------------:|
|         1         |         5         |      30      |      15      |
|         2         |        15         |      25      |      10      |
|         3         |        13         |      45      |      20      |
|        LF         |        \-         |      32      |      18      |
|        MRF        |        32         |      \-      |      15      |
|        WTE        |        18         |      15      |      \-      |

The fixed costs associated with the disposal options are incurred only
if the particular disposal option is implemented. The three cities
produce 100, 90, and 120 Mg/day of solid waste, respectively, with the
composition provided in the table below.

**Reminder**: Use `round(x; digits=n)` to report values to the
appropriate precision!

**In this problem**:

-   Formulate the waste load allocation problem and implement it in
    `JuMP`.
-   Draw a diagram showing the flows of waste between the cities and the
    facilities. Which facilities (if any) will not be used? Does this
    solution make sense?

In [3]:
#defining givens into variables

#facility fees and info 
facility = ["landfill", "MRF", "WTE"]
capacity = [200, 350, 210] #units of Mg
fixed_cost = [2000, 1500, 2500] #units of $/day
tipping_cost = [50,7,60] #units of $/Mg
recycling_cost = [0,40,0] #units of $/Mg
transport_cost = 1.5 #units of $/Mg

#distance breakdown
cities = ["City 1", "City 2", "City 3"]
city_1_distance = [5,30,15]
city_2_distance = [15,25,10]
city_3_distance = [13,45,20]
LF_distance = [0,32,18]
MRF_distance = [32,0,15]
WTE_distance = [18,15,0]

MRF_recycle = .4
ash_fraction = .16
reclycing_fraction = .14

0.14

Talk about decision variables... 


The objective of this model is to minimze costs: waste disposal and transportation costs. Costs inlcude fixed cost, tipping fee, recyling cost, and transport costs.

LF = 1, MRF = 2, WTE = 3

For landfill costs,

$2000 * Y_1 + 50 * (W_{1,1} + W_{2,1} + W_{3,1} + R_{2,1}+ R_{3,1})$

For MRF costs,

$1500 * Y_2 + 7 * (W_{1,2} + W_{2,2} + W_{3,2} + (.4 * 40 * ((W_{1,2} + W_{2,2} + W_{3,2}))))$

For Waste to Energy Costs,

$2500 * Y_3 + 60*(W_{1,3} + W_{2,3} + W_{3,3}+ R_{2,3})$

For transportation costs,

$1.5 * [5 * W_{1,1} + 30 * W_{1,2} + 15 * W_{1,3} + 15 * W_{2,1} + 25 * W_{2,2} + 10 * W_{2,3} + 13 * W_{3,1} + 45 * W_{3,2} + 20 * W_{3,3} + 32 * R_{2,1} + 18 * W_{3,1} + 15 * W_{2,3}]$

Constraints:

$ R_{2,3} + R_{2,1} = [W_{1,1} + W_{2,1} + W_{3,1}]*[1-0.4]$

$ R_{3,1} = [W_{1,1} + W_{2,1} + W_{3,1}]*(0.16) + R_{2,3} * (0.14)$

$ W_{1,1} + W_{2,1} + W_{3,1} + R_{2,1} + R_{3,1} <= 200$

$ W_{1,2} + W_{2,2} + W_{3,2} <= 350$

$ W_{1,3} + W_{2,3} + W_{3,3} + R_{2,3} <= 210$

If $\sum_{i\in I} W_{ij} = 0$

$ Y_j = 0 $

If $\sum_{i\in I} W_{ij} >= 0$

$ Y_j = 1 $


In [4]:
waste = Model(HiGHS.Optimizer);

I = 1:length(cities)
J = 1:length(facility)
K = 1:3

@variable(waste, W[i in I, j in J] >= 0);
@variable(waste, R[k in J, j in J] >= 0);
@variable(waste, Y[j in J], Bin);


@objective(waste, Min, sum(
@constraint(waste, commit2, !Y[2] => {W[1,2] + W[2,2] == 0}) 

Base.Meta.ParseError: ParseError:
# Error @ /Users/badynmilstein-touesnard/BEE 4750/hw5-ismabay/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X15sZmlsZQ==.jl:13:62
@objective(waste, Min, sum(
@constraint(waste, commit2, !Y[2] => {W[1,2] + W[2,2] == 0}) 
#                                                            └ ── Expected `)`

## References

List any external references consulted, including classmates.